# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
# show the first few rows of data for inspection
for row in full_data_rows_list[:3]:
    print(row)

['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39']
['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8']
["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8']


In [7]:
df = pd.read_csv('event_datafile_new.csv')
#  displaying first 5 rows of the DataFrame
print(df.head(5))

              artist firstName gender  itemInSession lastName     length  \
0            Des'ree    Kaylee      F              1  Summers  246.30812   
1            Mr Oizo    Kaylee      F              3  Summers  144.03873   
2         Tamba Trio    Kaylee      F              4  Summers  177.18812   
3     The Mars Volta    Kaylee      F              5  Summers  380.42077   
4  Infected Mushroom    Kaylee      F              6  Summers  440.26730   

  level                     location  sessionId                          song  \
0  free  Phoenix-Mesa-Scottsdale, AZ        139                  You Gotta Be   
1  free  Phoenix-Mesa-Scottsdale, AZ        139                       Flat 55   
2  free  Phoenix-Mesa-Scottsdale, AZ        139  Quem Quiser Encontrar O Amor   
3  free  Phoenix-Mesa-Scottsdale, AZ        139                     Eriatarka   
4  free  Phoenix-Mesa-Scottsdale, AZ        139               Becoming Insane   

   userId  
0       8  
1       8  
2       8  
3       

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 0 
- firstName of user 1
- gender of user 2
- item number in session 3
- last name of user 4
- length of the song 5
- level (paid or free song) 6
- location of the user 7
- sessionId 8
- song title 9
- userId 10

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
try:
    from cassandra.cluster import Cluster
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)  #catching any errors from cassandra cluster

#### Create Keyspace

In [9]:
# creating keyspace named sparkify
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
# connecting to sparkify keyspace
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
# we chose sessionId and itemInSession for our composite primary key to get unique
# rows to get correct information per the request from question 1

# creating song_info_by_session_and_item table specifically for the first query
try:    
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_info_by_session_and_item (
    sessionId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    length FLOAT,
    PRIMARY KEY (sessionId, itemInSession))
    """)
except Exception as e:
    print(e)
                    

we now have an empty table and ready to insert data

In [12]:
df = pd.read_csv('event_datafile_new.csv')

In [13]:
# referencing data to see column names and index
for index, column_name in enumerate(df.columns):
    print(f"{column_name}: {index}")

artist: 0
firstName: 1
gender: 2
itemInSession: 3
lastName: 4
length: 5
level: 6
location: 7
sessionId: 8
song: 9
userId: 10


In [15]:
# data from the CSV file
file = 'event_datafile_new.csv'
# processing contents in file and iterating through the lines in the CSV file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # creating the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session_and_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # adding the correct column elements for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

In [17]:
# select statement on song_info_by_session_and_item table for the first query
query = """
SELECT artist, song, length
FROM song_info_by_session_and_item 
WHERE sessionId=338 AND itemInSession=4
"""

try:
    session.row_factory = cassandra.query.dict_factory
    rows = session.execute(query)
except Exception as e:
    print(e)
# using Pandas dataframe to display query output
rows_list = list(rows)
df = pd.DataFrame.from_dict(data=rows_list)
df.head()

,artist,length,song
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


### For table user_song_playlist we used a composite partition key to improve overall performance

Previously, we were able to use userId and sessionId to optimize our queries performance; however, to optimize this queries performance we need to use the composite partition key as userId and sessionId along with a cluster key of itemInSession to achieve unique rows and proper performance. This narrows down the amount of data and operations to perform the query.

In [18]:
# creating user_song_playlist table specifically for the second query 
try:    
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_song_playlist (
    userId INT,
    sessionId INT,
    itemInSession INT,
    artist TEXT,
    firstName TEXT,
    lastName TEXT,
    song TEXT,
    PRIMARY KEY ( (userId, sessionId), itemInSession))
    """)
except Exception as e:
    print(e)                   

In [19]:
# data from the CSV file
file = 'event_datafile_new.csv'
# processing contents in file and iterating through the lines in the CSV file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # creating the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_playlist (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # adding the correct column elements for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

In [20]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user
## (first and last name) for userid = 10, sessionid = 182

In [22]:
# select statement on user_song_playlist table for the second query
query = """
SELECT artist, song, firstName, lastName
FROM user_song_playlist
WHERE userId = 10 AND sessionId = 182
ORDER BY itemInSession
"""

try:
    session.row_factory = cassandra.query.dict_factory
    rows = session.execute(query)
except Exception as e:
    print(e)
# using Pandas dataframe to display query output
rows_list = list(rows)
df = pd.DataFrame.from_dict(data=rows_list)
df.head()

,artist,firstname,lastname,song
0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
1,Three Drives,Sylvie,Cruz,Greece 2000
2,Sebastien Tellier,Sylvie,Cruz,Kilometer
3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


In [23]:
# creating user_song_history table specifically for the third query 
# the primary key has song as partition key and clustering column as userId for unique users
# this allows us to partition by all unique user song combinations
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_song_history (
    song TEXT,
    userId INT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY (song, userId) 
    )
    """)
except Exception as e:
    print(e)
                    

In [24]:
# data from the CSV file
file = 'event_datafile_new.csv'
# processing contents in file and iterating through the lines in the CSV file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # creating the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_history (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # adding the correct column elements for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#Query 3: Give me every user name (first and last) in my music app history
 who listened to the song 'All Hands Against His Own'


In [25]:
# select statement on user_song_history table for the third query
query = """
SELECT firstName, lastName
FROM user_song_history
WHERE song = 'All Hands Against His Own'
"""

try:
    session.row_factory = cassandra.query.dict_factory
    rows = session.execute(query)
except Exception as e:
    print(e)
# using Pandas dataframe to display query output
rows_list = list(rows)
df = pd.DataFrame.from_dict(data=rows_list)
df.head()

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [24]:
# dropping the tables before we close out the sessions
try:
    session.execute("DROP TABLE song_info_by_session_and_item")
    session.execute("DROP TABLE user_song_playlist")
    session.execute("DROP TABLE user_song_history")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()